# 3.Stage1
Access and report the target data via descriptive statistics
for a group of selected attributes to characterise the data and make a clear research
goal.

In [1]:
import pandas as pd
import warnings
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import scipy.cluster.hierarchy as sch

In [30]:
mydata = pd.read_feather("2018_sample.feather")

In [6]:
from sklearn.linear_model import LinearRegression

In [ ]:
import statsmodels.formula.api as sm

In [33]:
data = pd.concat((mydata, pd.get_dummies(mydata["Time_h"], drop_first=False)), axis=1)
data = pd.concat((mydata, pd.get_dummies(mydata["Weather Type"], drop_first=False)), axis=1)
data = pd.concat((data, pd.get_dummies(data["Type"], drop_first=False)), axis=1)

In [34]:
data

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,PULocationID,DOLocationID,fare_amount,tip_amount,total_amount,Date,...,Type,DATE,AWND,PRCP,SNOW,SNWD,TAVG,Weather Type,N,Y
0,2018-01-01 00:00:25,2018-01-01 00:06:08,1,1.36,43,237,6.5,2.34,10.14,2018-01-01,...,Holiday,2018-01-01,7.83,0.00,0.0,0.0,-10.555556,N,1,0
1,2018-01-01 00:03:46,2018-01-01 00:08:42,1,0.70,148,79,5.0,1.25,7.55,2018-01-01,...,Holiday,2018-01-01,7.83,0.00,0.0,0.0,-10.555556,N,1,0
2,2018-01-01 00:05:09,2018-01-01 00:09:03,3,0.90,263,43,5.5,1.35,8.15,2018-01-01,...,Holiday,2018-01-01,7.83,0.00,0.0,0.0,-10.555556,N,1,0
3,2018-01-01 00:08:03,2018-01-01 00:16:01,2,1.45,107,148,7.5,1.76,10.56,2018-01-01,...,Holiday,2018-01-01,7.83,0.00,0.0,0.0,-10.555556,N,1,0
4,2018-01-01 00:09:20,2018-01-01 00:39:38,1,2.60,162,48,19.0,4.05,24.35,2018-01-01,...,Holiday,2018-01-01,7.83,0.00,0.0,0.0,-10.555556,N,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
648293,2018-12-31 23:34:09,2018-12-31 23:40:50,1,0.69,249,249,6.0,1.46,8.76,2018-12-31,...,Workday,2018-12-31,0.00,1.02,0.0,0.0,5.555556,Y,0,1
648294,2018-12-31 23:35:31,2018-12-31 23:55:56,4,3.80,48,158,16.0,3.45,20.75,2018-12-31,...,Workday,2018-12-31,0.00,1.02,0.0,0.0,5.555556,Y,0,1
648295,2018-12-31 23:36:21,2018-12-31 23:44:45,1,2.40,142,151,9.0,2.50,12.80,2018-12-31,...,Workday,2018-12-31,0.00,1.02,0.0,0.0,5.555556,Y,0,1
648296,2018-12-31 23:44:40,2018-12-31 23:52:33,1,0.71,142,48,6.5,5.00,12.80,2018-12-31,...,Workday,2018-12-31,0.00,1.02,0.0,0.0,5.555556,Y,0,1


In [35]:
y = data["tip_amount"]
x = data.drop(columns=["tip_amount","tpep_pickup_datetime","tpep_dropoff_datetime","Date","DATE","Time_h","Weather Type","Type"])

In [41]:
x

,passenger_count,trip_distance,PULocationID,DOLocationID,fare_amount,total_amount,Druation,AWND,PRCP,SNOW,SNWD,TAVG,N,Y
0,1,1.36,43,237,6.5,10.14,5.716667,7.83,0.00,0.0,0.0,-10.555556,1,0
1,1,0.70,148,79,5.0,7.55,4.933333,7.83,0.00,0.0,0.0,-10.555556,1,0
2,3,0.90,263,43,5.5,8.15,3.900000,7.83,0.00,0.0,0.0,-10.555556,1,0
3,2,1.45,107,148,7.5,10.56,7.966667,7.83,0.00,0.0,0.0,-10.555556,1,0
4,1,2.60,162,48,19.0,24.35,30.300000,7.83,0.00,0.0,0.0,-10.555556,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
648293,1,0.69,249,249,6.0,8.76,6.683333,0.00,1.02,0.0,0.0,5.555556,0,1
648294,4,3.80,48,158,16.0,20.75,20.416667,0.00,1.02,0.0,0.0,5.555556,0,1
648295,1,2.40,142,151,9.0,12.80,8.400000,0.00,1.02,0.0,0.0,5.555556,0,1
648296,1,0.71,142,48,6.5,12.80,7.883333,0.00,1.02,0.0,0.0,5.555556,0,1


In [36]:
reg = LinearRegression().fit(x, y)

In [45]:
print(reg.coef_)
print(reg.score(x,y))

[-3.94631421e-04 -7.35549922e-02  8.11494421e-05 -2.32774466e-05
 -5.53671398e-01  5.87696592e-01  2.38778605e-02  1.50439335e-03
  3.39601955e-03  3.14328157e-03 -2.93831277e-03 -6.31109632e-04
  4.63746627e-03 -4.63746627e-03]
0.8366154023762589


In [ ]:
scipy.stats.anderson(data["tip_amount"])

In [ ]:
import scipy.stats as st
def get_best_distribution(data):
    dist_names = ["norm", "exponweib", "weibull_max", "weibull_min", "pareto", "genextreme"]
    dist_results = []
    params = {}
    for dist_name in dist_names:
        dist = getattr(st, dist_name)
        param = dist.fit(data)

        params[dist_name] = param
        # Applying the Kolmogorov-Smirnov test
        D, p = st.kstest(data, dist_name, args=param)
        print("p value for "+dist_name+" = "+str(p))
        dist_results.append((dist_name, p))

    # select the best fitted distribution
    best_dist, best_p = (max(dist_results, key=lambda item: item[1]))
    # store the name of the best fit and its p value

    print("Best fitting distribution: "+str(best_dist))
    print("Best p value: "+ str(best_p))
    print("Parameters for the best fit: "+ str(params[best_dist]))

    return best_dist, best_p, params[best_dist]

In [ ]:
get_best_distribution(mydata["tip_amount"][:10000])

In [ ]:
mydata["tip_amount"][:100]

In [ ]:
get_best_distribution(data["tip_amount"])

In [ ]:
import warnings
import numpy as np
import pandas as pd
import scipy.stats as st
import statsmodels as sm
import matplotlib
import matplotlib.pyplot as plt

matplotlib.rcParams['figure.figsize'] = (16.0, 12.0)
matplotlib.style.use('ggplot')

# Create models from data
def best_fit_distribution(data, bins=200, ax=None):
    """Model data by finding best fit distribution to data"""
    # Get histogram of original data
    y, x = np.histogram(data, bins=bins, density=True)
    x = (x + np.roll(x, -1))[:-1] / 2.0

    # Distributions to check
    DISTRIBUTIONS = [        
        st.alpha,st.beta,st.chi,st.chi2,
        st.dgamma,st.dweibull,st.expon,st.exponnorm,st.exponweib,st.exponpow,st.f,
        st.gamma,st.gengamma,st.invgamma,st.invgauss,
        st.invweibull,st.loggamma,st.loglaplace,st.lognorm,st.lomax,st.norm,st.pareto
    ]

    # Best holders
    best_distribution = st.norm
    best_params = (0.0, 1.0)
    best_sse = np.inf

    # Estimate distribution parameters from data
    for distribution in DISTRIBUTIONS:

        # Try to fit the distribution
        try:
            # Ignore warnings from data that can't be fit
            with warnings.catch_warnings():
                warnings.filterwarnings('ignore')

                # fit dist to data
                params = distribution.fit(data)

                # Separate parts of parameters
                arg = params[:-2]
                loc = params[-2]
                scale = params[-1]

                # Calculate fitted PDF and error with fit in distribution
                pdf = distribution.pdf(x, loc=loc, scale=scale, *arg)
                sse = np.sum(np.power(y - pdf, 2.0))

                # if axis pass in add to plot
                try:
                    if ax:
                        pd.Series(pdf, x).plot(ax=ax)
                    end
                except Exception:
                    pass

                # identify if this distribution is better
                if best_sse > sse > 0:
                    best_distribution = distribution
                    best_params = params
                    best_sse = sse

        except Exception:
            pass

    return (best_distribution.name, best_params)

def make_pdf(dist, params, size=10000):
    """Generate distributions's Probability Distribution Function """

    # Separate parts of parameters
    arg = params[:-2]
    loc = params[-2]
    scale = params[-1]

    # Get sane start and end points of distribution
    start = dist.ppf(0.01, *arg, loc=loc, scale=scale) if arg else dist.ppf(0.01, loc=loc, scale=scale)
    end = dist.ppf(0.99, *arg, loc=loc, scale=scale) if arg else dist.ppf(0.99, loc=loc, scale=scale)

    # Build PDF and turn into pandas Series
    x = np.linspace(start, end, size)
    y = dist.pdf(x, loc=loc, scale=scale, *arg)
    pdf = pd.Series(y, x)

    return pdf

# Load data from statsmodels datasets
data = mydata["tip_amount"][:100]

# Plot for comparison
plt.figure(figsize=(12,8))
ax = data.plot(kind='hist', bins=50, alpha=0.5)
# Save plot limits
dataYLim = ax.get_ylim()

# Find best fit distribution
best_fit_name, best_fit_params = best_fit_distribution(data, 200, ax)
best_dist = getattr(st, best_fit_name)

# Update plots
ax.set_ylim(dataYLim)
ax.set_title(u'El Niño sea temp.\n All Fitted Distributions')
ax.set_xlabel(u'Temp (°C)')
ax.set_ylabel('Frequency')

# Make PDF with best params 
pdf = make_pdf(best_dist, best_fit_params)

# Display
plt.figure(figsize=(12,8))
ax = pdf.plot(lw=2, label='PDF', legend=True)
data.plot(kind='hist', bins=50, alpha=0.5, label='Data', legend=True, ax=ax)

param_names = (best_dist.shapes + ', loc, scale').split(', ') if best_dist.shapes else ['loc', 'scale']
param_str = ', '.join(['{}={:0.2f}'.format(k,v) for k,v in zip(param_names, best_fit_params)])
dist_str = '{}({})'.format(best_fit_name, param_str)

ax.set_title(u'El Niño sea temp. with best fit distribution \n' + dist_str)
ax.set_xlabel(u'Temp. (°C)')
ax.set_ylabel('Frequency')